In [10]:
#Import
import time
import mysql.connector
import os
import requests as r
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [11]:
#Training function
def model_train(zones):
    start= time.time()
    val_list = []
    validator = open("validation.txt", "r")
    for line in validator:
        val_list.append(line.rstrip())

    mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )
    for i in zones:
        #Get random interval
        
        print("Receiving number interval for zone {0}".format(i))
        
        mycursor = mydb.cursor()
        sql = """SELECT COUNT(*) as nr FROM team12dublinbus.final where zone = """ +str(1)+""";"""
        mycursor.execute(sql)
        result_list = []
        rand_no = 0
        for (r) in mycursor:
            rand_no = str(round(150000/int(r[0]), 4))
            
        print("Received number interval from DB for zone {0}".format(i))
        print("Receiving dataframe for zone {0}".format(i))
        
        mycursor = mydb.cursor()
        sql = """SELECT `time_to_sec(m.hr_time)`,travel_time,precipIntensity,temperature,humidity,dist,holiday,weekend 
        FROM team12dublinbus.final
        WHERE zone = """+str(i)+""" and RAND() < """+str(rand_no)+""";"""
        mycursor.execute(sql)
        
        print("Received dataframe for zone {0}. Processing frame...".format(i))
        
        result_list = []
        for (l) in mycursor:
            result_list.extend([l])
        
        frame = DataFrame.from_records(result_list)
        frame.columns = ['time_of_day', 'time','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']
        
        X= frame[['time_of_day','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']]
        Y = frame[['time']]
                  
        #Model and scaler declaration
        print("Training model and scalers for zone {0}".format(i))
        model = SVR()
        feat_scaler = StandardScaler()
        tar_scaler = StandardScaler()

        # Transform data
        X = feat_scaler.fit_transform(X[['time_of_day','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']])         
        Y = tar_scaler.fit_transform(Y[['time']])
        print("Scalers fitted for zone {0}, beginning model train".format(i))
        model.fit(X,Y)
        print("Model trained for zone {0}, beginning export".format(i))         
        #Export
        #Model
        model_file = 'model_zone_{0}.sav'.format(i)
        pickle.dump(model, open(model_file, 'wb'))          
        #Target
        tar_file = "tar_scaler_{0}.sav".format(i)
        pickle.dump(tar_scaler, open(tar_file, 'wb'))

        #Feature
        feat_file = "feat_scaler_{0}.sav".format(i)
        pickle.dump(feat_scaler, open(feat_file, 'wb'))
        
        print("Completed operations for Zone {0}".format(i))
        print(model)
    print(time.time() - start)
        

In [13]:

model_train(["1","2","3","4","5","6","7","8","9","10","11","12"])

Receiving number interval for zone 1
Received number interval from DB for zone 1
Receiving dataframe for zone 1
Received dataframe for zone 1. Processing frame...
Training model and scalers for zone 1
Scalers fitted for zone 1, beginning model train


E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

Model trained for zone 1, beginning export
Completed operations for Zone 1
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
Receiving number interval for zone 2
Received number interval from DB for zone 2
Receiving dataframe for zone 2
Received dataframe for zone 2. Processing frame...
Training model and scalers for zone 2


E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

Scalers fitted for zone 2, beginning model train
Model trained for zone 2, beginning export
Completed operations for Zone 2
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
Receiving number interval for zone 3
Received number interval from DB for zone 3
Receiving dataframe for zone 3
Received dataframe for zone 3. Processing frame...
Training model and scalers for zone 3
Scalers fitted for zone 3, beginning model train


...

E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

Received number interval from DB for zone 11
Receiving dataframe for zone 11
Received dataframe for zone 11. Processing frame...
Training model and scalers for zone 11
Scalers fitted for zone 11, beginning model train
Model trained for zone 11, beginning export
Completed operations for Zone 11
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
Receiving number interval for zone 12


E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

Received number interval from DB for zone 12
Receiving dataframe for zone 12
Received dataframe for zone 12. Processing frame...
Training model and scalers for zone 12
Scalers fitted for zone 12, beginning model train
Model trained for zone 12, beginning export
Completed operations for Zone 12
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
18840.6816842556


E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\envs\env\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
E:\Anaconda\envs\env\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 